# Investigating RNNs and RL using the N-back cognitive task

**NMA 2023 Group 1 Project**

__Content creators:__ Aland Astudillo, Campbell Border, Disheng, Julia Yin, Koffivi, Rishabh Mallik, Shuwen Liu, Zelin Zhang

__Pod TA:__ Suryanarayanan Nagar Anthel Venkatesh

__Project Mentor:__ 

---
# Objective

- 

- 
---

# Project Design
---

# Setup

## Install Dependencies

In [1]:
# @title Install dependencies
!pip install jedi --quiet
!pip install --upgrade pip setuptools wheel --quiet
!pip install numpy==1.23.3 --quiet --ignore-installed
!pip install gymnasium --quiet


#!pip install dm-acme[jax] --quiet
#!pip install dm-sonnet --quiet
#!pip install trfl --quiet
#!pip uninstall seaborn -y --quiet
#!pip install seaborn --quiet

In [2]:
# @title Imports

#import time
import numpy as np
#import pandas as pd
#import matplotlib.pyplot as plt
#import seaborn as sns
import gymnasium as gym
from gymnasium import spaces

## Figure settings

In [ ]:
# @title Figure settings
from IPython.display import clear_output, display, HTML
%matplotlib inline
# sns.set()

---
# Background

## Replace with our own literature review

- Cognitive scientists use standard lab tests to tap into specific processes in the brain and behavior. Some examples of those tests are Stroop, N-back, Digit Span, TMT (Trail making tests), and WCST (Wisconsin Card Sorting Tests).

- Despite an extensive body of research that explains human performance using descriptive what-models, we still need a more sophisticated approach to gain a better understanding of the underlying processes (i.e., a how-model).

- Interestingly, many of such tests can be thought of as a continuous stream of stimuli and corresponding actions, that is in consonant with the RL formulation. In fact, RL itself is in part motivated by how the brain enables goal-directed behaviors using reward systems, making it a good choice to explain human performance.

- One behavioral test example would be the N-back task.

  - In the N-back, participants view a sequence of stimuli, one by one, and are asked to categorize each stimulus as being either match or non-match. Stimuli are usually numbers, and feedback is given at both timestep and trajectory levels.

  - The agent is rewarded when its response matches the stimulus that was shown N steps back in the episode. A simpler version of the N-back uses two-choice action schema, that is match vs non-match. Once the present stimulus matches the one presented N step back, then the agent is expected to respond to it as being a `match`.


- Given a trained RL agent, we then find correlates of its fitted parameters with the brain mechanisms. The most straightforward composition could be the correlation of model parameters with the brain activities.

## Datasets

- Human Connectome Project Working Memory (HCP WM) task ([NMA-CN HCP notebooks](https://github.com/NeuromatchAcademy/course-content/tree/master/projects/fMRI))

1200 subjects, each subject experience 8 blocks of 2-back and 8 blocks of 0-back.

## N-back Tasks

In the N-back task, participants view a sequence of stimuli, one per time, and are asked to categorize each stimulus as being either match or non-match. Stimuli are usually numbers, and feedbacks are given at both timestep and trajectory levels.

In a typical neuro setup, both accuracy and response time are measured, but here, for the sake of brevity, we focus only on accuracy of responses.

- 2 back working memory task:

The second condition is a 2-Back condition. During such a block, the subject is presented with a sequence of 10 images and must respond if each image is identical to the one 2 positions earlier or not (figure, right). At the beginning of the block there is a cue screen informing the subject that the upcoming stimuli are part of the 2-Back protocol. The timing of the cue screen, the presentation of the 10 stimulus images and of the response interval are identical to that of the 0-Back condition.

- 0-back control memory task:

The first is a match-to-sample condition (termed in the following text as 0-Back) during which a cue “Target” image is presented at the beginning of a block and which the subject has been instructed to memorize. Then a sequence of 10 images is presented. 


Any dataset that used cognitive tests would work.
Question: limit to behavioral data vs fMRI?
Question: Which stimuli and actions to use?
classic tests can be modeled using 1) bounded symbolic stimuli/actions (e.g., A, B, C), but more sophisticated one would require texts or images (e.g., face vs neutral images in social stroop dataset)
The HCP dataset from NMA-CN contains behavioral and imaging data for 7 cognitive tests including various versions of N-back.


Details of the tMEG Working Memory task

Working memory is assessed using an N-back task in which participants are asked to monitor
sequentially presented pictures. Participants are presented with blocks of trials that consisted of
pictures of tools or faces. Within each run, the 2 different stimulus types are presented in
separate blocks. Also, within each run, ½ of the blocks use a 2-back working memory task and
½ use a 0-back working memory task (as a working memory comparison). Participants are
instructed to press a button for every picture. If the currently presented picture matches the
cued picture (0-Back) or the same picture that was presented two pictures before (2-Back),
subjects press one button with their right index finger. For non-matching pictures, participants
press a second button with their right middle finger. Two runs are performed, 16 blocks each,
with a bright fixation "rest" on dark background for 15 seconds between blocks. 

- Special modelling of images (we are probably not going to model this)

There are 2 different categories of images used in this experiment: images of faces and tools.
Each block contains images from a single category. Some of the images in the non-matched
trials have been characterized as “Lure”. These images have been selected so that they have
common features with the target image, but are still different. These trials as flagged as “Lure”.
I


https://www.humanconnectome.org/storage/app/media/documentation/s1200/HCP_S1200_Release_Reference_Manual.pdf Page 72

---
## Implementation scheme

### Environment

The following cell implments N-back envinronment, that we later use to train a RL agent on human data. It is capable of performing two kinds of simulation:
- rewards the agent once the action was correct (i.e., a normative model of the environment).
- receives human data (or mock data if you prefer), and returns what participants performed as the observation. This is more useful for preference-based RL.

In [ ]:
# @title Define environment

# N-back environment
class NBack(gym.Env):

    # N = 2
    # step_count =        [ 0  1  2  3  4  5  6 ]
    # sequence =          [ a  b  c  d  a  d  a ]
    # correct actions =   [ ~  ~  0  0  0  1  1 ]

    # actions =           [ ~  ~  1  0  0  1  0 ]
    # reward_class =      [ ~  ~  FP TN TN TP FN]
    # reward =            [ ~  ~  -1  0  0  1 -1]

  ACTIONS = ['match', 'non-match']

  # Rewards input is structured as (TP, TN, FP, FN) (positive being matches)
  def __init__(self, 
               N=2, 
               num_steps=10,
               num_blocks=8, 
               num_memory = 5,
               stimulus_choices = list(string.ascii_lowercase),
               human_data = none):

    # basic setting
    self.N = N # N in N-back task
    self.num_steps = num_steps # how many image showed per block
    self.num_blocks = num_blocks # how many blocks per participant
    self.stimulus_choices = stimulus_choices # what stimuli are available
    self.num_stim = len(stimulus_choices)
    self.num_memory = num_memory
    self.step_count = 0

    # Define rewards, observation space and action space
    self.observations = np.zeros(shape = (num_stim, num_memory)).astype('int')  # will be filled in the `reset()`
    self.reward_range = spaces.Discrete(2)                                      # Range of rewards based on inputs
    self.action_space = spaces.Discrete(2)                                      # 0 (No match) or 1 (Match)
    
    self._action_history = []

    # copy and pasted from template 
    # whether mimic humans or reward the agent once it responds optimally.
    if human_data is None:
      self._imitate_human = False
      self.human_data = None
      self.human_subject_data = None
    else:
      self._imitate_human = True
      self.human_data = human_data
      self.human_subject_data = None


  def reset(self, seed=None):

    # Seed RNG
    super().reset(seed=seed)

    # Generate sequence of length self.episode_length, using one hot encoding
    observations = np.zeros(shape = (self.num_stim, self.num_memory)).astype('int')
    for i in range(self.num_memory):
      observations[random.randrange(self.num_stim), i] = 1

    # Generate correct sequence of actions
    correct_actions = [None] * self.N + [int(self.sequence[i] == self.sequence[i + self.N]) for i in range(self.episode_length - self.N)]

    # clearing the action history and reset step count
    self._action_history = []
    self.step_count = 0

    self.observations = observations
    self.correct_actions = correct_actions

    return observations

  
  
  def step(self, action):

    #
    self.step_count += 1

    # get reward
    reward
    
    # update observations
    
    # update action space
    self._action_history.append(action)



    # Get reward
    if self.step_count >= self.N:
      if (self.correct_actions[self.step_count]): # Match
        reward = self.rewards[0] if action else self.rewards[3] # TP if matched else FN
      else: # No match
        reward = self.rewards[2] if action else self.rewards[1] # FP if matches else TN
    else:
      reward = None

    # Get next character in sequence (or end episode)
    self.step_count += 1
    if self.step_count < self.episode_length:
      observation = self.sequence[self.step_count]
      done = False
    else:
      observation = None
      done = True
    info = "Not sure"

    return observation, reward, done, info



In [ ]:
# @title Test environment

env = NBack()
print(f"First char is {env.reset()[0]}")
print(env.sequence)
print(env.correct_actions)
for i in range(16):
  print(env.step(i % 2))


### Define a random agent

In [ ]:
# Random agent
class RandomAgent(nn.Module):
    
    def __init__(self, env):
        super().__init__()
        self.env = env

    # Choose a random action, 0 or 1
    def choose_action(self, seq):
        return np.random.randint(0, 2)

    def test(self, num_episodes):
        
        # Arrays to hold true/false positives and false negatives
        tps = np.zeros(num_episodes)
        fps = np.zeros_like(tps)
        fns = np.zeros_like(tps)

        for i in range(num_episodes):
            
            # Array of actions
            actions = np.zeros(self.env.episode_length, dtype=int)

            # Perform action for each element in sequence
            seq, step_count, _, done = self.env.reset()
            while not done:
                action = self.choose_action(seq)
                actions[step_count] = action
                seq, step_count, _, done = self.env.step(action)
            
            # Get episode data
            actions = actions[self.env.N:]
            correct_actions = self.env.correct_actions
            tps[i] = np.dot(actions, correct_actions) 
            fps[i] = np.dot(actions, 1 - correct_actions)
            fns[i] = np.dot(1 - actions, correct_actions)

        return tps, fps, fns

### Define a simple Q-learning agent

In [ ]:
class QLearningMLP(nn.Module):
    
    def __init__(self, env, input_size=1, hidden_sizes=[], actv="ReLU()"):
        super().__init__()

        self.env = env                 
        self.input_size = input_size
        self.output_size = 2
        self.hidden_sizes = hidden_sizes
        self.mlp = nn.Sequential()

        # Create net
        prev_size = self.input_size # Initialize the temporary input feature to each layer
        for i in range(len(hidden_sizes)): # Loop over layers and create each one
            
            # Add linear layer
            current_size = hidden_sizes[i] # Assign the current layer hidden unit from list
            layer = nn.Linear(prev_size, current_size)
            prev_size = current_size # Assign next layer input using current layer output
            self.mlp.add_module('Linear_%d'%i, layer) # Append layer to the model

            # Add activation function
            actv_layer = eval('nn.%s'%actv) # Assign activation function (eval allows us to instantiate object from string)
            self.mlp.add_module('Activation_%d'%i, actv_layer) # Append activation to the model with a name

        out_layer = nn.Linear(prev_size, self.output_size) # Create final layer
        self.mlp.add_module('Output_Linear', out_layer) # Append the final layer

        # Softmax layer
        softmax_layer = nn.Softmax()
        self.mlp.add_module('Output_Softmax', softmax_layer)

    def forward(self, x):

        # Do we need to reshape?
        input = torch.Tensor(x)
        return self.mlp(input)

    def choose_action(self, seq):

        # Run sequence through network
        output = self.forward(seq)
        # Choose highest likelihood action
        return np.argmax(output.detach())


    def train(self, num_episodes):

        for i in range(num_episodes):

            # Array of actions
            actions = np.zeros(self.env.episode_length, dtype=int)

            # Reset environment
            seq, step_count, _, done = self.env.reset()

            while not done:

                # Choose action and recieve reward
                action = self.choose_action(seq)
                actions[step_count] = action
                seq, step_count, reward, done = self.env.step(action)

                # Use reward to update network



### Define a Recurrent Deep Q-learning Agent (RDQN)

### Attempt to define a RNN in pytorch

In [ ]:
"""
Model definition

https://medium.com/@VersuS_/coding-a-recurrent-neural-network-rnn-from-scratch-using-pytorch-a6c9fc8ed4a7

https://www.youtube.com/watch?v=WEV61GmmPrk
"""

class RNN(nn.Module):
    """
    Basic RNN block. This represents a single layer of RNN
    """
    def __init__(self, input_size: int, hidden_size: int, output_size: int) -> None:
        """
        input_size: Number of features of your input vector
        hidden_size: Number of hidden neurons
        output_size: Number of features of your output vector
        """
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.batch_size = batch_size

        # input to hidden
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size, bias=False)
        # input to output
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        # softmax layer
        self.softmax = nn.LogSoftmax(dim = 1)

    
    def forward(self, input_tensor, hidden_tensor) -> tuple[torch.Tensor, torch.Tensor]:
        """
        Returns softmax(linear_out) and tanh(i2h + i2o)
        Inputs
        ------
        x: Input vector x  with shape (vocab_size, )
        hidden_state: Hidden state matrix
        Outputs
        -------
        out: Prediction vector
        hidden_state: New hidden state matrix
        """
        combined = torch.cat((input_tensor, output_tensor), 1)

        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)

        return output, hidden
        

    def init_hidden(self, batch_size=1) -> torch.Tensor:
        """
        Returns a hidden state with specified batch size. Defaults to 1
        """
        return torch.zeros(1, self.hidden_size)

In [ ]:
# training RNN layer

criterion = nn.NLLLoss()
learning_rate = 0.005
optimizer = torch.optim.SGD(rnn.parameters(), lr = learning_rate)

def train(input_tensor, action_tensor):
    hidden = rnn.init_hidden()
    for i in range(input_tensor.size()[0]):
        output, hidden = rnn(input_tensor[i], hidden)


# Section 4: Model(s)

In [2]:
# Random agent
class RandomAgent(nn.Module):
    
    

SyntaxError: unexpected EOF while parsing (3220473413.py, line 4)

In [ ]:
# RNN

class LayeredRNN(nn.Module)